Task
1. Create a RAG pipeline that can take following text and answer following questions
2. Try different types of chunking to get better answers?
3. Does asking questions differently give better answers? Why?
4. Try a different similarity search instead of cosine similarity - do the answers improve?



In [ ]:
sample_text = """
The Amazon rainforest is the largest tropical rainforest in the world, covering approximately 5.5 million square kilometers. It spans across nine countries, including Brazil, Peru, and Colombia. The rainforest is home to around 10% of the known species on Earth, including jaguars, sloths, and thousands of species of insects and birds.

Deforestation is a significant threat to the Amazon, with thousands of square kilometers lost each year due to agriculture, logging, and urbanization. This deforestation contributes to climate change, as the rainforest acts as a major carbon sink, absorbing millions of tons of carbon dioxide annually.

Indigenous tribes have lived in the Amazon for thousands of years, relying on its rich biodiversity for food, medicine, and shelter. These tribes have unique languages, traditions, and knowledge of the ecosystem. However, many face threats from illegal land encroachment and industrial activities.

Scientists believe that the Amazon plays a crucial role in global weather patterns by releasing water vapor into the atmosphere, which influences rainfall across South America and even other continents. The Amazon River, which flows through the rainforest, is the second longest river in the world and carries more water than any other river.

Efforts to protect the Amazon include international agreements, conservation programs, and sustainable development projects that aim to balance economic growth with environmental protection. Many organizations and governments are working to reduce illegal logging and promote reforestation initiatives.
"""

In [ ]:
questions = [
    "What is the Amazon rainforest?",
    "Which countries does the Amazon span across?",
    "Why is deforestation a problem in the Amazon?",
    "How does the Amazon rainforest affect global weather patterns?",
    "What role do indigenous tribes play in the Amazon?",
    "What is the importance of the Amazon River?",
    "What types of wildlife can be found in the Amazon?",
    "How does deforestation contribute to climate change?",
    "What efforts are being made to protect the Amazon?",
    "Why is the Amazon considered a major carbon sink?"
]

### Imports

In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import re

1. Create a RAG pipeline that can take following text and answer following questions


In [27]:
# Load pre-trained Sentence Transformer model
MODEL_NAME = "sentence-transformers/paraphrase-MiniLM-L6-v2"
model = SentenceTransformer(MODEL_NAME)

# Encode the sample text
sample_text = """
The Amazon rainforest is the largest tropical rainforest in the world, covering approximately 5.5 million square kilometers. It spans across nine countries, including Brazil, Peru, and Colombia. The rainforest is home to around 10% of the known species on Earth, including jaguars, sloths, and thousands of species of insects and birds.

Deforestation is a significant threat to the Amazon, with thousands of square kilometers lost each year due to agriculture, logging, and urbanization. This deforestation contributes to climate change, as the rainforest acts as a major carbon sink, absorbing millions of tons of carbon dioxide annually.

Indigenous tribes have lived in the Amazon for thousands of years, relying on its rich biodiversity for food, medicine, and shelter. These tribes have unique languages, traditions, and knowledge of the ecosystem. However, many face threats from illegal land encroachment and industrial activities.

Scientists believe that the Amazon plays a crucial role in global weather patterns by releasing water vapor into the atmosphere, which influences rainfall across South America and even other continents. The Amazon River, which flows through the rainforest, is the second longest river in the world and carries more water than any other river.

Efforts to protect the Amazon include international agreements, conservation programs, and sustainable development projects that aim to balance economic growth with environmental protection. Many organizations and governments are working to reduce illegal logging and promote reforestation initiatives.
"""

In [7]:
# Function to split text into meaningful Chunks
def split_text(text):
    return [para.strip() for para in re.split("\n+", text) if para.strip()]  # Split by newlines and periods

# Store document changes
stored_text = split_text(sample_text)   # Split the sample text into sentences
stored_embeddings = model.encode(stored_text)  # Encode the sentences

In [8]:
stored_text

['The Amazon rainforest is the largest tropical rainforest in the world, covering approximately 5.5 million square kilometers. It spans across nine countries, including Brazil, Peru, and Colombia. The rainforest is home to around 10% of the known species on Earth, including jaguars, sloths, and thousands of species of insects and birds.',
 'Deforestation is a significant threat to the Amazon, with thousands of square kilometers lost each year due to agriculture, logging, and urbanization. This deforestation contributes to climate change, as the rainforest acts as a major carbon sink, absorbing millions of tons of carbon dioxide annually.',
 'Indigenous tribes have lived in the Amazon for thousands of years, relying on its rich biodiversity for food, medicine, and shelter. These tribes have unique languages, traditions, and knowledge of the ecosystem. However, many face threats from illegal land encroachment and industrial activities.',
 'Scientists believe that the Amazon plays a cruci

In [11]:
# Function to generate embeddings
def get_transformer_embedding(text):
    return model.encode(text, convert_to_numpy=True)

# Function to retrieve relevant text
def retrieve_passage(query):
    query_embedding = get_transformer_embedding([query])
    similarities = cosine_similarity(query_embedding, stored_embeddings)[0] 
    best_match_idx = np.argmax(similarities)
    return stored_text[best_match_idx]

# Function to answer questions, based on stored content
def answer_question(query):
    relevant_passage = retrieve_passage(query)
    return relevant_passage

In [13]:
# Sample questions and Answers
questions = [
    "What is the Amazon rainforest?",
    "Which countries does the Amazon span across?",
    "Why is deforestation a problem in the Amazon?",
    "How does the Amazon rainforest affect global weather patterns?",
    "What role do indigenous tribes play in the Amazon?",
    "What is the importance of the Amazon River?",
    "What types of wildlife can be found in the Amazon?",
    "How does deforestation contribute to climate change?",
    "What efforts are being made to protect the Amazon?",
    "Why is the Amazon considered a major carbon sink?"
]

print("\nSample Questions and Answers:\n")
for question in questions:
    response = answer_question(question)
    print(f"Q: {question}\nA: {response}\n")


Sample Questions and Answers:

Q: What is the Amazon rainforest?
A: The Amazon rainforest is the largest tropical rainforest in the world, covering approximately 5.5 million square kilometers. It spans across nine countries, including Brazil, Peru, and Colombia. The rainforest is home to around 10% of the known species on Earth, including jaguars, sloths, and thousands of species of insects and birds.

Q: Which countries does the Amazon span across?
A: The Amazon rainforest is the largest tropical rainforest in the world, covering approximately 5.5 million square kilometers. It spans across nine countries, including Brazil, Peru, and Colombia. The rainforest is home to around 10% of the known species on Earth, including jaguars, sloths, and thousands of species of insects and birds.

Q: Why is deforestation a problem in the Amazon?
A: Deforestation is a significant threat to the Amazon, with thousands of square kilometers lost each year due to agriculture, logging, and urbanization. T

2. Try different types of chunking to get better answers?


In [17]:
# Alternative form of chunking

# Function to split text into meaningful Chunks
def split_text(text):
    return [sentence.strip() for sentence in re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text) if sentence.strip()]  # Split by newlines and periods


# Store document changes
stored_text = split_text(sample_text)   # Split the sample text into sentences
stored_embeddings = model.encode(stored_text)  # Encode the sentences

stored_text

['The Amazon rainforest is the largest tropical rainforest in the world, covering approximately 5.5 million square kilometers.',
 'It spans across nine countries, including Brazil, Peru, and Colombia.',
 'The rainforest is home to around 10% of the known species on Earth, including jaguars, sloths, and thousands of species of insects and birds.',
 'Deforestation is a significant threat to the Amazon, with thousands of square kilometers lost each year due to agriculture, logging, and urbanization.',
 'This deforestation contributes to climate change, as the rainforest acts as a major carbon sink, absorbing millions of tons of carbon dioxide annually.',
 'Indigenous tribes have lived in the Amazon for thousands of years, relying on its rich biodiversity for food, medicine, and shelter.',
 'These tribes have unique languages, traditions, and knowledge of the ecosystem.',
 'However, many face threats from illegal land encroachment and industrial activities.',
 'Scientists believe that the 

In [18]:
# function to generate embeddings
def get_transformer_embedding(text):
    return model.encode(text, convert_to_numpy=True)

# Function to retrieve relevant passage
def retrieve_passage(query):
    query_embedding = get_transformer_embedding([query])
    similarities = cosine_similarity(query_embedding, stored_embeddings)[0] 
    best_match_idx = np.argmax(similarities)
    return stored_text[best_match_idx]

# Function to answer questions, based on stored content
def answer_question(query):
    relevant_passage = retrieve_passage(query)
    return relevant_passage

In [20]:
# Sample questions and Answers

questions = [
    "What is the Amazon rainforest?",
    "Which countries does the Amazon span across?",
    "Why is deforestation a problem in the Amazon?",
    "How does the Amazon rainforest affect global weather patterns?",
    "What role do indigenous tribes play in the Amazon?",
    "What is the importance of the Amazon River?",
    "What types of wildlife can be found in the Amazon?",
    "How does deforestation contribute to climate change?",
    "What efforts are being made to protect the Amazon?",
    "Why is the Amazon considered a major carbon sink?"
]

print("\nSample Questions and Answers:\n")
for question in questions:
    response = answer_question(question)
    print(f"Q: {question}\nA: {response}\n")


Sample Questions and Answers:

Q: What is the Amazon rainforest?
A: The Amazon rainforest is the largest tropical rainforest in the world, covering approximately 5.5 million square kilometers.

Q: Which countries does the Amazon span across?
A: Efforts to protect the Amazon include international agreements, conservation programs, and sustainable development projects that aim to balance economic growth with environmental protection.

Q: Why is deforestation a problem in the Amazon?
A: Deforestation is a significant threat to the Amazon, with thousands of square kilometers lost each year due to agriculture, logging, and urbanization.

Q: How does the Amazon rainforest affect global weather patterns?
A: Scientists believe that the Amazon plays a crucial role in global weather patterns by releasing water vapor into the atmosphere, which influences rainfall across South America and even other continents.

Q: What role do indigenous tribes play in the Amazon?
A: Indigenous tribes have lived

3. Does asking questions differently give better answers? Why?


In [ ]:
# Trying to ask the questions in a different way

questions = [
    "What is the Amazon rainforest?",
    "Which countries does the Amazon span across?",
    "Why is deforestation a problem in the Amazon?",
    "How does the Amazon rainforest affect global weather patterns?",
    "What role do indigenous tribes play in the Amazon?",
    "What is the importance of the Amazon River?",
    "What types of wildlife can be found in the Amazon?",
    "How does deforestation contribute to climate change?",
    "What efforts are being made to protect the Amazon?",
    "Why is the Amazon considered a major carbon sink?"
]

Asking different questions can be beneficial in the sense that some words are weighted higher.

We already removed the english stop words, so they are not interfering. But trying to ask questions with words might have a higher weight than other words, you might be able to get more precise answers.

Example of this could be the question "Which countries does the Amazon span across?" instead of asking an open question, you could try to specify the question to "Does the Amazon run through Brazil?". It might be the case that "Brazil" have a higher weight in the model, compared to "span" and "countries".

4. Try a different similarity search instead of cosine similarity - do the answers improve?


In [35]:
# Tryin a different similarity search instead of cosine similarity
from sklearn.metrics.pairwise import euclidean_distances

# Function to split text into meaningful Chunks
def split_text(text):
    return [sentence.strip() for sentence in re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text) if sentence.strip()]  # Split by newlines and periods

# Store document changes
stored_text = split_text(sample_text)   # Split the sample text into sentences
stored_embeddings = model.encode(stored_text)  # Encode the sentences

stored_text

['The Amazon rainforest is the largest tropical rainforest in the world, covering approximately 5.5 million square kilometers.',
 'It spans across nine countries, including Brazil, Peru, and Colombia.',
 'The rainforest is home to around 10% of the known species on Earth, including jaguars, sloths, and thousands of species of insects and birds.',
 'Deforestation is a significant threat to the Amazon, with thousands of square kilometers lost each year due to agriculture, logging, and urbanization.',
 'This deforestation contributes to climate change, as the rainforest acts as a major carbon sink, absorbing millions of tons of carbon dioxide annually.',
 'Indigenous tribes have lived in the Amazon for thousands of years, relying on its rich biodiversity for food, medicine, and shelter.',
 'These tribes have unique languages, traditions, and knowledge of the ecosystem.',
 'However, many face threats from illegal land encroachment and industrial activities.',
 'Scientists believe that the 

In [37]:
# Function to generate embeddings
def get_transformer_embedding(text):
    return model.encode(text, convert_to_numpy=True)

# Function to retrieve relevant passage
def retrieve_passage(query):
    query_embedding = get_transformer_embedding([query])
    similarities = euclidean_distances(query_embedding, stored_embeddings)[0] 
    best_match_idx = np.argmin(similarities)
    return stored_text[best_match_idx]

# Function to answer questions, based on stored content
def answer_question(query):
    relevant_passage = retrieve_passage(query)
    return relevant_passage

In [38]:
# Sample questions and Answers
questions = [
    "What is the Amazon rainforest?",
    "Which countries does the Amazon span across?",
    "Why is deforestation a problem in the Amazon?",
    "How does the Amazon rainforest affect global weather patterns?",
    "What role do indigenous tribes play in the Amazon?",
    "What is the importance of the Amazon River?",
    "What types of wildlife can be found in the Amazon?",
    "How does deforestation contribute to climate change?",
    "What efforts are being made to protect the Amazon?",
    "Why is the Amazon considered a major carbon sink?"
]

print("\nSample Questions and Answers:\n")
for question in questions:
    response = answer_question(question)
    print(f"Q: {question}\nA: {response}\n")


Sample Questions and Answers:

Q: What is the Amazon rainforest?
A: The Amazon rainforest is the largest tropical rainforest in the world, covering approximately 5.5 million square kilometers.

Q: Which countries does the Amazon span across?
A: Efforts to protect the Amazon include international agreements, conservation programs, and sustainable development projects that aim to balance economic growth with environmental protection.

Q: Why is deforestation a problem in the Amazon?
A: Deforestation is a significant threat to the Amazon, with thousands of square kilometers lost each year due to agriculture, logging, and urbanization.

Q: How does the Amazon rainforest affect global weather patterns?
A: Scientists believe that the Amazon plays a crucial role in global weather patterns by releasing water vapor into the atmosphere, which influences rainfall across South America and even other continents.

Q: What role do indigenous tribes play in the Amazon?
A: Indigenous tribes have lived